In [ ]:
%load_ext autoreload

import sys
import os
import pickle as pk

import numpy as np
nax = np.newaxis
import pandas as pd

from pathlib import Path 
path = Path("../spqubolib").resolve().as_posix()
sys.path.append(path)
if True:
    from spqubolib.qubo import qubo
    from spqubolib.solver import MA_solver

from blob_experiment import build_q, generate_problem, timer
from blob_plot import plot_problem, plot_answer, plot_answer_2_scatter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
flag_plot = True
flag_run = False
result_path = "run"
image_path = "images/blob"

STAGE = os.environ.get("STAGE", "")
print("STAGE ==", STAGE)
if STAGE=="run":
    flag_plot = False
    flag_run = True
    os.makedirs(result_path, exist_ok=True)
elif STAGE=="plot":
    flag_plot = True
    flag_run = False
    os.makedirs(image_path, exist_ok=True)

# Problem 1

## setting parameters

In [ ]:
#### setting parameters ####

# problem size
B, D, K, Nk = 51, 2, 7, 100 # = (B, D, K, M)
# problem parameter
Ainte, Acnt = 1, 50 # C = 50

# solver parameter
num_steps = 1000 # T_0 = 1000
p_const1 = 0.5 # p_0 = 0.5
p_const2_relative = 1
beta0 = 1 # \beta_0 = 1
c_const1_relative = 0.5 # theta_1 = 1 / 0.5 = 2
seed = 0

## make problem and seriarize

In [ ]:
%autoreload

if flag_run:
    rng = np.random.default_rng(0)
    B_list = np.array([B] * D)
    df_prob = generate_problem(rng, K, B_list, Nk)

    num_points = df_prob.shape[0]
    num_spins = num_points * K
    print(f"points: {num_points}")
    print(f"spins: {num_spins}")
    print(f"spins**2: {num_spins**2}")

    ret = []
    for k in range(K):
        df_ = df_prob.copy()
        df_["k"] = k
        ret.append(df_)
    ret = pd.concat(ret, axis=0)

    q, _, _ = build_q(B, D, K, ret[list(np.arange(D)) + ["k"]].values, Ainte, Acnt)
    q.set_mode("fourier")

    with open(f"{result_path}/blob_problem.pk", "wb") as f:
        pk.dump(df_prob, file=f)

## load problem

In [ ]:
%autoreload

with open(f"{result_path}/blob_problem.pk", "rb") as f:
    df_prob = pk.load(f)

if flag_plot:
    plot_problem(df_prob, B, f"{image_path}/blob_problem.pdf", subcaption="a")

## solve by momentum annealing

In [ ]:
%autoreload

if flag_run:
    prob_f = qubo(q, mode="min", vartype="binary")

    for label, prob in [("fourier", prob_f)]:
        solver = MA_solver(
            K = num_steps,
            save_ignore=["I_log"],
            p_const1 = p_const1,  # 0.5
            p_const2 = num_steps * p_const2_relative,  # 1000
            beta0 = beta0,  # 1
            c_const1 = num_steps * c_const1_relative,  # 500
            seed = seed,
            filename_log=f"{result_path}/MA_log_blob.npz"
        )

        with timer() as t:
            ans, L_ans = prob.solve(solver)
        elapsed = t.t

        np.savez(f"{result_path}/blob_answer.npz",
            ans = ans,
            K = K,
            Nk = Nk,
            label = label)

## plot results

### plot answer

In [ ]:
%autoreload

def load_and_plot_answer():
    data = np.load(f"{result_path}/blob_answer.npz")
    ans = data["ans"]
    K = data["K"]
    Nk = data["Nk"]
    label = data["label"]
    plot_answer(ans, K, Nk, f"{image_path}/blob_answer.png")

if flag_plot:
    load_and_plot_answer()

In [ ]:
%autoreload

def load_and_plot_answer_2():
    data = np.load(f"{result_path}/blob_answer.npz")
    ans = data["ans"]
    K = data["K"]
    Nk = data["Nk"]
    label = data["label"]
    plot_answer_2_scatter(df_prob, B, ans, K, Nk, f"{image_path}/blob_answer_2.pdf", subcaption="b")

if flag_plot:
    load_and_plot_answer_2()

/Users/hiroshi_yamashita/GoogleDrive/kenkyu/ising/fourier_publish_20231102/spqubo_experiments/clustering/blob_plot.py:96: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  
